# Datasets and Data reduction 

Stuffs for data reduction and datasets preprocessing

## Compile APOKACS-2

The `.txt` can be downloaded here: https://iopscience.iop.org/article/10.3847/1538-4365/aaebfd#apjsaaebfdt5

In [2]:
from astropy.io import ascii
from py import apokasc2_path

apokasc2 = ascii.read(apokasc2_path)
apokasc2 = apokasc2[
    (~apokasc2["Nmax"].mask)
    & (~apokasc2["Dnu"].mask)
    & (~apokasc2["M(cor)"].mask)
    & (apokasc2["E_LogAge"] < 1)
    & (apokasc2["E_LogAge"] > 0)
    & ["SeisUnc" not in i for i in apokasc2["Notes"]]
]
apokasc2

KIC,2MASS,Teff,e_Teff,FeH,e_FeH,AFe,e_AFe,Nmax,e_Nmax,Dnu,e_Dnu,ES,Fdnu,e_Fdnu,M(cor),e_M(cor)-ran,e_M(cor)-sys,R(cor),e_R(cor)-ran,e_R(cor)-sys,logg(seis),e_logg(seis)-ran,e_logg(seis)-sys,Rho,e_Rho-ran,e_Rho-sys,LogAge,E_LogAge,e_LogAge,Av,e_Av,Notes
,,K,K,dex(---),dex(---),dex(---),dex(---),uHz,,uHz,,,,,Msun,,,Rsun,,,dex(cm / s2),,,g / cm3,,,Myr,Myr,Myr,mag,mag,
int32,str18,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str18
1027337,2M19252021+3647118,4636.0,67.3,0.275,0.024,0.023,0.01,73.975,0.009,6.991,0.013,RGB,1.0333,0.0003,1.227,0.063,0.038,7.544,0.029,0.014,2.772,0.005,0.007,0.004029,0.026,0.007,3.798,0.099,-0.09,0.214,0.071,...
1160789,2M19233280+3652207,4729.6,72.3,-0.257,0.034,0.188,0.015,25.209,0.015,3.545,0.011,RC,0.9965,0.0324,0.875,0.147,0.082,10.86,0.071,0.025,2.308,0.007,0.01,0.0009635,0.069,0.008,3.889,0.133,-0.146,0.009,0.074,...
1161447,2M19241746+3651460,4776.1,86.2,0.058,0.029,-0.006,0.013,37.066,0.027,4.153,0.011,RC,1.003,0.0238,1.46,0.135,0.076,11.54,0.059,0.022,2.478,0.012,0.01,0.00134,0.052,0.008,3.396,0.166,-0.145,0.4,0.086,...
1161618,2M19242614+3648478,4742.0,72.1,0.064,0.029,0.005,0.012,33.926,0.01,4.093,0.012,RC,1.001,0.0033,1.183,0.063,0.077,10.879,0.028,0.023,2.438,0.005,0.01,0.001296,0.026,0.008,3.639,0.064,-0.069,0.199,0.074,...
1162220,2M19245791+3653298,4190.1,51.7,0.083,0.021,0.07,0.011,11.0,0.01,1.669,0.011,RGB,1.0484,0.0004,1.007,0.055,0.044,18.175,0.024,0.017,1.922,0.005,0.007,0.0002364,0.021,0.008,4.056,0.087,-0.083,0.181,0.073,...
1162746,2M19252639+3649116,4798.1,75.6,-0.388,0.038,0.229,0.017,27.798,0.015,3.763,0.01,RC,0.9972,0.0281,0.941,0.131,0.08,10.688,0.062,0.024,2.354,0.007,0.01,0.001087,0.06,0.008,3.786,0.139,-0.131,0.172,0.075,...
1163114,2M19254564+3650475,4285.8,54.4,0.297,0.02,0.025,0.01,14.356,0.011,1.887,0.008,RGB,1.0396,0.0004,1.467,0.05,0.043,19.097,0.021,0.016,2.042,0.006,0.007,0.0002971,0.016,0.008,3.538,0.079,-0.077,0.382,0.069,...
1163359,2M19255838+3650557,4571.9,71.0,-0.339,0.032,0.218,0.017,21.468,0.009,2.632,0.009,RGB,1.0346,0.0005,1.454,0.051,0.043,15.297,0.022,0.016,2.231,0.005,0.007,0.0005725,0.018,0.008,3.376,0.075,-0.072,0.195,0.076,...


In [3]:
# Add TIC cross-match to APOKASC-2 and then cross-match to allstar DR17

import h5py
import numpy as np
from astropy.io import fits
from astroNN.apogee import allstar
from py import apokasc2_h5_path, apokasc2_pipeline_path

# cross matching pipeline
apokasc2_pipeline = ascii.read(apokasc2_pipeline_path)
apokasc2_pipeline = apokasc2_pipeline.filled(-1.)
xy, x_ind, y_ind = np.intersect1d(
    apokasc2_pipeline["KIC"], apokasc2["KIC"], return_indices=True
)
apokasc2_pipeline = apokasc2_pipeline[x_ind]

# =============== matching to APOGEE DR14 ===============
allstar_f = fits.getdata(allstar(dr=14))
# cross matching with allstar by APOGEE_ID
xy, x_ind, y_ind = np.intersect1d(
    apokasc2["2MASS"], allstar_f["APOGEE_ID"], return_indices=True
)
# make sure x_ind is in ascending order
apogeeidx = y_ind[np.argsort(x_ind)]

# make sure every entry has one match
assert len(x_ind) == len(apokasc2)
apokasc2.add_column(apogeeidx, name="allstar_dr14_idx")
# =============== matching to APOGEE DR14 ===============

# =============== matching to APOGEE DR17 ===============
allstar_f = fits.getdata(allstar(dr=17))
# cross matching with allstar by APOGEE_ID
xy, x_ind, y_ind = np.intersect1d(
    apokasc2["2MASS"], allstar_f["APOGEE_ID"], return_indices=True
)
# make sure x_ind is in ascending order
apogeeidx = y_ind[np.argsort(x_ind)]

# make sure every entry has one match
assert len(x_ind) == len(apokasc2)
apokasc2.add_column(apogeeidx, name="allstar_dr17_idx")
# =============== matching to APOGEE DR17 ===============

# process evolutionary state
evostate = apokasc2["ES"].astype("S")
evostate_int = np.zeros(len(evostate), dtype=int)
for indx, t in enumerate(evostate):
    if "rgb" in t.lower():
        evostate_int[indx] = 1
    elif "rc" in t.lower() or "clump" in t.lower():
        evostate_int[indx] = 2
    else:
        evostate_int[indx] = -1

h5f = h5py.File(apokasc2_h5_path, "w")
h5f.create_dataset("KIC", data=np.array(apokasc2["KIC"]).astype(int))
h5f.create_dataset("allstar_dr14_idx", data=apokasc2["allstar_dr14_idx"])
h5f.create_dataset("allstar_dr17_idx", data=apokasc2["allstar_dr17_idx"])
h5f.create_dataset("Numax", data=apokasc2["Nmax"])
h5f.create_dataset("Numax_err", data=apokasc2["e_Nmax"])
h5f.create_dataset("Deltanu", data=apokasc2["Dnu"])
h5f.create_dataset("Deltanu_err", data=apokasc2["e_Dnu"])
h5f.create_dataset("Mass", data=apokasc2["M(cor)"])
h5f.create_dataset("Mass_err", data=np.sqrt((apokasc2["M(cor)"]*apokasc2["e_M(cor)-ran"])**2 + (apokasc2["M(cor)"]*apokasc2["e_M(cor)-sys"])**2))
h5f.create_dataset("R", data=apokasc2["R(cor)"])
h5f.create_dataset("R_err", data=np.sqrt((apokasc2["R(cor)"]*apokasc2["e_R(cor)-ran"])**2 + (apokasc2["R(cor)"]*apokasc2["e_R(cor)-sys"])**2))
h5f.create_dataset("Age", data=10**apokasc2["LogAge"] / 1000)
h5f.create_dataset(
    # have checked this formula consistent with the one used in APOKASC-3
    "Age_err", data=(10**(apokasc2["LogAge"] + apokasc2["E_LogAge"]) - 10**apokasc2["LogAge"]) / 1000
)
h5f.create_dataset("LogAge", data=apokasc2["LogAge"]-3)
h5f.create_dataset(
    # have checked this formula too in Gyr
    "LogAge_err", data=apokasc2["E_LogAge"]
)
# -1=unidentified, 1=RGB, 2=RC
h5f.create_dataset("ES", data=evostate_int)

# ============== individual pipeline ==============
h5f.create_dataset("Numax_A2Z", data=apokasc2_pipeline["Nmax-A2Z"])
h5f.create_dataset("Numax_A2Z_err", data=apokasc2_pipeline["e_Nmax-A2Z"])
h5f.create_dataset("Deltanu_A2Z", data=apokasc2_pipeline["Dnu-A2Z"])
h5f.create_dataset("Deltanu_A2Z_err", data=apokasc2_pipeline["e_Dnu-A2Z"])

h5f.create_dataset("Numax_CAN", data=apokasc2_pipeline["Nmax-CAN"])
h5f.create_dataset("Numax_CAN_err", data=apokasc2_pipeline["e_Nmax-CAN"])
h5f.create_dataset("Deltanu_CAN", data=apokasc2_pipeline["Dnu-CAN"])
h5f.create_dataset("Deltanu_CAN_err", data=apokasc2_pipeline["e_Dnu-CAN"])

h5f.create_dataset("Numax_COR", data=apokasc2_pipeline["Nmax-COR"])
h5f.create_dataset("Numax_COR_err", data=apokasc2_pipeline["e_Nmax-COR"])
h5f.create_dataset("Deltanu_COR", data=apokasc2_pipeline["Dnu-COR"])
h5f.create_dataset("Deltanu_COR_err", data=apokasc2_pipeline["e_Dnu-COR"])

h5f.create_dataset("Numax_OCT", data=apokasc2_pipeline["Nmax-OCT"])
h5f.create_dataset("Numax_OCT_err", data=apokasc2_pipeline["e_Nmax-OCT"])
h5f.create_dataset("Deltanu_OCT", data=apokasc2_pipeline["Dnu-OCT"])
h5f.create_dataset("Deltanu_OCT_err", data=apokasc2_pipeline["e_Dnu-OCT"])

h5f.create_dataset("Numax_SYD", data=apokasc2_pipeline["Nmax-SYD"])
h5f.create_dataset("Numax_SYD_err", data=apokasc2_pipeline["e_Nmax-SYD"])
h5f.create_dataset("Deltanu_SYD", data=apokasc2_pipeline["Dnu-SYD"])
h5f.create_dataset("Deltanu_SYD_err", data=apokasc2_pipeline["e_Dnu-SYD"])
# ============== individual pipeline ==============

h5f.close()

## Download all lightcurves

Please run it as a script as it involves multiprocessing

In [ ]:
!python ./py/download_apokasc_lightcurves.py

## Generate Kepler Light Curves for APOKASC

Please run it as a script as it involves multiprocessing

In [ ]:
!python ./py/gen_apokasc_lightcurves.py

## Generate Kepler Power Spectrum for APOKASC

Please run it as a script as it involves multiprocessing

In [ ]:
!python ./py/gen_apokasc_powerspec.py

## Compile Yu et al. 2018 

In [ ]:
import numpy as np
from astroNN.apogee import allstar
from astropy.io import fits, ascii
from astropy.table import Table, hstack
from astroNN.datasets.xmatch import xmatch

# table1.dat and table2.dat from https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJS/236/42#/browse
yu_etal_2018 = ascii.read("./yu_etal_2018/table1.dat")
yu_etal_2018.remove_column("KIC")
yu_etal_2018 = hstack([ascii.read("./yu_etal_2018/table2.dat"), yu_etal_2018])

# search.csv manually from https://archive.stsci.edu/kepler/kepler_fov/search.php?form=fuf
kic_xmatch = ascii.read("./yu_etal_2018/search.csv")[1:]
yu_etal_2018.add_columns([kic_xmatch["RA (J2000)"].astype(float), kic_xmatch["Dec (J2000)"].astype(float)], names=["RA", "DEC"])

f_allstar = fits.getdata(allstar(dr=17))

idx1, idx2, sep = xmatch(f_allstar["RA"][1:], f_allstar["DEC"][1:], yu_etal_2018["RA"], yu_etal_2018["DEC"], maxdist=4)
yu_etal_2018 = yu_etal_2018[idx2]

# remember to pad 1 because we discarded the first row of allstar
# indices location of yu 2018 in allstar dr17
yu_etal_2018.add_columns([idx1+1], names=["allstar_dr17_idx"])

yu_etal_2018.write("./data_files/yu_etal_2018_apogeedr17.dat", format="ascii")